In [1]:
import numpy as np
import os
import keras
import tempfile
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input
from tensorflow_model_optimization.sparsity import keras as sparsity

"""physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)"""

'physical_devices = tf.config.experimental.list_physical_devices(\'GPU\')\nassert len(physical_devices) > 0, "Not enough GPU hardware devices available"\nconfig = tf.config.experimental.set_memory_growth(physical_devices[0], True)'

In [2]:
model = tf.keras.models.load_model('finalmodel.h5')

In [3]:
nsfw_images = [fn for fn in os.listdir(r'D:\Data\Train\NSFW') if fn.endswith('.jpg')]
sfw_images = [fn for fn in os.listdir(r'D:\Data\Train\SFW') if fn.endswith('.jpg')]
train_dir = r'D:\Data\Train'
validation_dir = r'D:\Data\Validation'

"""def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)"""

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory(train_dir,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=30,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator =test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 30,
                                                          color_mode='rgb',
                                                          class_mode  = 'categorical', 
                                                          target_size = (224, 224),
                                                          shuffle=True)

Found 6394 images belonging to 2 classes.
Found 2137 images belonging to 2 classes.


In [4]:
epochs = 5
end_step = np.ceil(1.0 * 6394 / 30).astype(np.int32) * epochs
print(end_step)

1070


In [5]:
new_pruning_parameters = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                final_sparsity=0.90,
                                                begin_step=0,
                                                end_step=end_step,
                                                frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(model, **new_pruning_parameters)
new_pruned_model.summary()

Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
prune_low_magnitude_conv1_pa (None, 225, 225, 3)       1         
_________________________________________________________________
prune_low_magnitude_conv1 (P (None, 112, 112, 32)      1730      
_________________________________________________________________
prune_low_magnitude_conv1_bn (None, 112, 112, 32)      129       
_________________________________________________________________
prune_low_magnitude_conv1_re (None, 112, 112, 32)      1         
_________________________________________________________________
prune_low_magnitude_conv_dw_ (None, 112, 112, 32)      289       
_______________________________________________

In [6]:
new_pruned_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00000001),
    metrics=['accuracy']
)

In [7]:
callback_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, 
                                                          min_lr=0.00000001, cooldown=1)

logdir = tempfile.mkdtemp()

callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0),
    callback_early_stopping,
    callback_reduce_lr
]

new_pruned_model.fit(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=int(6394/30),
                    validation_steps=int(2137/30),
                    epochs = epochs,
                    callbacks = callbacks)

Epoch 1/5
184/213 [========================>.....] - ETA: 40s - loss: 0.1081 - accuracy: 0.9589

C:\Users\Harshinee Sriram\anaconda3\lib\site-packages\PIL\Image.py:2832: DecompressionBombWarning: Image size (147659204 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


213/213 [==============================] - 393s 2s/step - loss: 0.1159 - accuracy: 0.9554 - val_loss: 1.9695 - val_accuracy: 0.4906
Epoch 2/5
213/213 [==============================] - 391s 2s/step - loss: 0.1553 - accuracy: 0.9408 - val_loss: 2.7471 - val_accuracy: 0.4906
Epoch 3/5
213/213 [==============================] - 382s 2s/step - loss: 0.4776 - accuracy: 0.8363 - val_loss: 0.9211 - val_accuracy: 0.4901
Epoch 4/5
213/213 [==============================] - 381s 2s/step - loss: 0.4672 - accuracy: 0.7382 - val_loss: 1.9607 - val_accuracy: 0.4906
Epoch 5/5
213/213 [==============================] - 380s 2s/step - loss: 0.2506 - accuracy: 0.8982 - val_loss: 0.3354 - val_accuracy: 0.8596


In [8]:
"""final_model = sparsity.strip_pruning(new_pruned_model)
final_model.summary()"""

'final_model = sparsity.strip_pruning(new_pruned_model)\nfinal_model.summary()'

In [9]:
"""final_model.save('compressed.h5')
model = load_model('compressed.h5')
for i, w in enumerate(model.get_weights()):
    print(
        "{} -- Total:{}, Zeros: {:.2f}%".format(
            model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100
        )
    )"""

'final_model.save(\'compressed.h5\')\nmodel = load_model(\'compressed.h5\')\nfor i, w in enumerate(model.get_weights()):\n    print(\n        "{} -- Total:{}, Zeros: {:.2f}%".format(\n            model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100\n        )\n    )'